In [1]:
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.linear_model import LogisticRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats.stats import pearsonr
from scipy.stats import norm

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import (
    RandomForestClassifier,
    ExtraTreesClassifier,
    AdaBoostClassifier,
    GradientBoostingClassifier,
)
from sklearn.model_selection import cross_validate

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score

## Loading Data

In [2]:
with open('../data/X_train.npy', 'rb') as f:
    X_train = np.load(f)
with open('../data/X_test.npy', 'rb') as f:
    X_test = np.load(f)
with open('../data/y_train.npy', 'rb') as f:
    y_train = np.load(f)
with open('../data/y_test.npy', 'rb') as f:
    y_test = np.load(f)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(13972, 29) (6872, 29) (13972,) (6872,)


In [3]:
with open('X_train2.npy', 'rb') as f:
    X_train2 = np.load(f)
with open('X_test2.npy', 'rb') as f:
    X_test2 = np.load(f)
with open('X_train3.npy', 'rb') as f:
    X_train3 = np.load(f)
with open('X_test3.npy', 'rb') as f:
    X_test3 = np.load(f)

In [4]:
columns = ['Administrative',
 'Administrative_Duration',
 'Informational',
 'Informational_Duration',
 'ProductRelated',
 'ProductRelated_Duration',
 'BounceRates',
 'ExitRates',
 'PageValues',
 'SpecialDay',
 'OperatingSystems',
 'Browser',
 'Region',
 'TrafficType',
 'Month_Aug',
 'Month_Dec',
 'Month_Feb',
 'Month_Jul',
 'Month_June',
 'Month_Mar',
 'Month_May',
 'Month_Nov',
 'Month_Oct',
 'Month_Sep',
 'VisitorType_New_Visitor',
 'VisitorType_Other',
 'VisitorType_Returning_Visitor',
 'Weekend_False',
 'Weekend_True',
 'Revenue']

feature_names = columns
feature_names.remove("Revenue")

In [5]:
X_train = pd.DataFrame(X_train, columns=feature_names)
X_train = X_train.drop("Weekend_False",axis=1)
X_test = pd.DataFrame(X_test, columns=feature_names)
X_test = X_test.drop("Weekend_False",axis=1)

In [6]:
X_DF = pd.concat([X_train,X_test])
y = np.concatenate((y_train,y_test))

## Tuning Hyperparameters

In [7]:
model = GradientBoostingClassifier()

In [8]:
param_dist = {
    "n_estimators": list(np.linspace(200, 1600, 8, dtype=int)),
    "learning_rate": [0.15, 0.1, 0.05, 0.01, 0.005, 0.001],
    "max_features": ["auto", "sqrt"],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_depth": [4, 6, 8, None],
}

In [9]:
def get_best_params(model,param_dist,X,y):
    model_rand = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=100, cv=3, verbose=2, random_state=0, n_jobs=-1)
    model_rand.fit(X,y)
    best_params = model_rand.best_params_
    return best_params

In [10]:
best_params = get_best_params(model,param_dist,X_train,y_train)
print(best_params)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
{'n_estimators': 1400, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': None, 'learning_rate': 0.005}


## Fitting the Model

In [11]:
model_best = GradientBoostingClassifier(**best_params)
model_best.fit(X_train,y_train) 

GradientBoostingClassifier(learning_rate=0.005, max_depth=None,
                           max_features='sqrt', n_estimators=1400)

In [12]:
score = cross_val_score(model_best, X_train, y_train)
print(np.round(score.mean(), 4))

0.9203


In [13]:
def get_performance(model,X,y):
    y_pred = model.predict(X)
    target_names = ['0: No Revenue', '1: Revenue']
    report = classification_report(y_test, y_pred, target_names=target_names)
    
    print(report)
    print("Confusion Matrix:\n",confusion_matrix(y,y_pred))
    print("Accuracy: ",accuracy_score(y,y_pred))
    print("ROC-AUC: ",roc_auc_score(y,model.predict_proba(X)[:,1]))

In [14]:
get_performance(model_best,X_test,y_test)

               precision    recall  f1-score   support

0: No Revenue       0.91      0.95      0.93      3436
   1: Revenue       0.94      0.91      0.93      3436

     accuracy                           0.93      6872
    macro avg       0.93      0.93      0.93      6872
 weighted avg       0.93      0.93      0.93      6872

Confusion Matrix:
 [[3252  184]
 [ 305 3131]]
Accuracy:  0.9288416763678696
ROC-AUC:  0.9822820346370214


## Model with Feature Engineering

In [15]:
model2 = GradientBoostingClassifier(**best_params)
model2.fit(X_train2, y_train)

GradientBoostingClassifier(learning_rate=0.005, max_depth=None,
                           max_features='sqrt', n_estimators=1400)

In [16]:
get_performance(model2,X_test2,y_test)

               precision    recall  f1-score   support

0: No Revenue       0.93      0.94      0.93      3436
   1: Revenue       0.94      0.92      0.93      3436

     accuracy                           0.93      6872
    macro avg       0.93      0.93      0.93      6872
 weighted avg       0.93      0.93      0.93      6872

Confusion Matrix:
 [[3226  210]
 [ 261 3175]]
Accuracy:  0.9314610011641443
ROC-AUC:  0.9838436007974186


## Model with Feature Selection and Feature Engineering

In [17]:
model3 = GradientBoostingClassifier(**best_params)
model3.fit(X_train3, y_train)

GradientBoostingClassifier(learning_rate=0.005, max_depth=None,
                           max_features='sqrt', n_estimators=1400)

In [18]:
get_performance(model3,X_test3,y_test)

               precision    recall  f1-score   support

0: No Revenue       0.93      0.94      0.93      3436
   1: Revenue       0.94      0.93      0.93      3436

     accuracy                           0.93      6872
    macro avg       0.93      0.93      0.93      6872
 weighted avg       0.93      0.93      0.93      6872

Confusion Matrix:
 [[3224  212]
 [ 252 3184]]
Accuracy:  0.9324796274738067
ROC-AUC:  0.983464051113933
